# Quantum Volume Optimizer

## A World-Class Deep Agent Example

This notebook demonstrates a sophisticated multi-agent system that finds the optimal **Quantum Volume (QV)** configuration for any IBM Quantum backend. It showcases the power of combining all Qiskit MCP servers with LangChain's Deep Agents framework.

### What is Quantum Volume?

Quantum Volume is a single-number metric that captures the largest random circuit of equal width and depth that a quantum computer can successfully implement. A QV of 2^n means the device can reliably execute n-qubit circuits of depth n.

Key factors affecting QV:
- **Two-qubit gate fidelity** (most important)
- **Qubit connectivity** (linear chains need SWAP gates)
- **Coherence times** (T1, T2)
- **Readout accuracy**

## Architecture

```
                    QUANTUM VOLUME OPTIMIZER
                      (Coordinator Agent)
                             |
          +------------------+------------------+
          |                  |                  |
          v                  v                  v
   BACKEND ANALYST    QUBIT CHAIN        TRANSPILER
                      OPTIMIZER          BENCHMARKER
          |                  |                  |
          v                  v                  v
   qiskit-ibm-        qiskit-ibm-       qiskit-ibm-
   runtime-mcp        runtime-mcp       transpiler-mcp
   (backends)         (QV qubit tools)  + qiskit-mcp
```

The **Qubit Chain Optimizer** now uses algorithmic tools:
- `find_optimal_qv_qubits_tool`: Finds densely connected subgraphs for QV
- `find_optimal_qubit_chains_tool`: Finds optimal linear chains
- `get_coupling_map_tool`: Gets backend connectivity graph

## Setup

```bash
pip install deepagents langchain langchain-mcp-adapters python-dotenv
pip install langchain-anthropic
pip install qiskit-mcp-servers
```

In [ ]:
import os

from deepagents import create_deep_agent
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langchain_mcp_adapters.client import MultiServerMCPClient


load_dotenv()

print("Configuration:")
print(f"  QISKIT_IBM_TOKEN: {'Set' if os.getenv('QISKIT_IBM_TOKEN') else 'Not set'}")
print(f"  ANTHROPIC_API_KEY: {'Set' if os.getenv('ANTHROPIC_API_KEY') else 'Not set'}")

In [ ]:
# System prompts
COORDINATOR_PROMPT = """You are the Quantum Volume Optimizer coordinating specialized agents.

Your team:
1. backend-analyst: IBM Quantum backend expert (list backends, get properties)
2. qubit-chain-optimizer: Uses algorithmic tools (find_optimal_qv_qubits_tool) to find best qubits
3. transpiler-benchmarker: Circuit optimization expert (compare local vs AI transpilation)

Produce a report with: Executive Summary, Backend Analysis, Optimal Qubit Subsets,
Transpilation Comparison, and QV Recommendation."""

BACKEND_ANALYST_PROMPT = """You are the Backend Analyst. List backends, get properties,
find suitable systems for QV experiments."""

QUBIT_CHAIN_PROMPT = """You are the Qubit Chain Optimizer with access to algorithmic tools:
- find_optimal_qv_qubits_tool: Find densely connected subgraphs for QV (use this!)
- find_optimal_qubit_chains_tool: Find optimal linear chains
- get_coupling_map_tool: Get backend connectivity

For QV experiments, use find_optimal_qv_qubits_tool with metric='qv_optimized'."""

TRANSPILER_PROMPT = """You are the Transpiler Benchmarker. Compare local vs AI-powered
optimization, minimize circuit depth and two-qubit gates."""

In [ ]:
def get_mcp_config():
    return {
        "qiskit-ibm-runtime": {
            "transport": "stdio",
            "command": "qiskit-ibm-runtime-mcp-server",
            "args": [],
            "env": {
                "QISKIT_IBM_TOKEN": os.getenv("QISKIT_IBM_TOKEN", ""),
                "QISKIT_IBM_RUNTIME_MCP_INSTANCE": os.getenv("QISKIT_IBM_RUNTIME_MCP_INSTANCE", ""),
            },
        },
        "qiskit": {
            "transport": "stdio",
            "command": "qiskit-mcp-server",
            "args": [],
            "env": {},
        },
        "qiskit-ibm-transpiler": {
            "transport": "stdio",
            "command": "qiskit-ibm-transpiler-mcp-server",
            "args": [],
            "env": {"QISKIT_IBM_TOKEN": os.getenv("QISKIT_IBM_TOKEN", "")},
        },
    }


def generate_qv_qasm(num_qubits: int, depth: int | None = None, seed: int = 42) -> str:
    """Generate a true Quantum Volume circuit using Qiskit's library."""
    from qiskit.circuit.library import quantum_volume
    from qiskit.qasm3 import dumps

    qv_circuit = quantum_volume(num_qubits, depth=depth, seed=seed)
    return dumps(qv_circuit.decompose())

In [ ]:
async def create_agent():
    mcp_config = get_mcp_config()
    mcp_client = MultiServerMCPClient(mcp_config)

    # Load tools using get_tools() which creates self-managing tools
    # that handle their own sessions (new session per tool call)
    all_tools, server_tools = [], {}
    for name in mcp_config:
        try:
            tools = await mcp_client.get_tools(server_name=name)
            server_tools[name] = tools
            all_tools.extend(tools)
            print(f"{name}: {len(tools)} tools")
        except Exception as e:
            print(f"{name}: FAILED - {e}")

    llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0, max_tokens=8192)

    subagents = [
        {
            "name": "backend-analyst",
            "description": "IBM Quantum backend expert",
            "system_prompt": BACKEND_ANALYST_PROMPT,
            "tools": server_tools.get("qiskit-ibm-runtime", []),
        },
        {
            "name": "qubit-chain-optimizer",
            "description": "Topology analysis expert with algorithmic qubit finding tools",
            "system_prompt": QUBIT_CHAIN_PROMPT,
            "tools": server_tools.get("qiskit-ibm-runtime", []),  # Has QV qubit tools
        },
        {
            "name": "transpiler-benchmarker",
            "description": "Circuit optimization expert",
            "system_prompt": TRANSPILER_PROMPT,
            "tools": server_tools.get("qiskit", []) + server_tools.get("qiskit-ibm-transpiler", []),
        },
    ]

    return create_deep_agent(
        model=llm, tools=all_tools, system_prompt=COORDINATOR_PROMPT, subagents=subagents
    )


agent = await create_agent()
print("\nAgent ready!")

In [ ]:
# Run optimization
request = f"""
Find the optimal Quantum Volume configuration for my IBM Quantum account.

1. Discover available backends (use backend-analyst)
2. Use find_optimal_qv_qubits_tool to find optimal qubit subsets for QV depths 2-5
3. Compare local vs AI transpilation for the best qubit configurations
4. Generate recommendation report with specific qubits and expected QV

Sample QV-4 circuit for transpilation testing:
```qasm
{generate_qv_qasm(4)}
```
"""

result = await agent.ainvoke({"messages": [{"role": "user", "content": request}]})
print(result.get("messages", [])[-1].content if result.get("messages") else "No response")

In [ ]:
# Interactive follow-up
while True:
    query = input("You: ").strip()
    if query.lower() in ["quit", "exit", "q"]:
        break
    if not query:
        continue
    result = await agent.ainvoke({"messages": [{"role": "user", "content": query}]})
    print(
        f"\nAssistant: {result.get('messages', [])[-1].content if result.get('messages') else 'No response'}\n"
    )